## Install Packages

In [3]:
#pip install pydealer

## Import Packages

In [4]:
import pydealer
import random

## Deck and Hand Creation

In [5]:
deck = pydealer.Deck()
deck.shuffle()

newDeck = []

for i in deck:
  newDeck.append(i.value)

user_hand = random.sample(newDeck, 7)

for i in user_hand:
  newDeck.remove(i)

agent_hand = random.sample(newDeck, 7)

for i in agent_hand:
  newDeck.remove(i)

## Variable Checks

In [6]:
# To ensure that user starts first
isUserTurn = False

# To ensure that the user/agent cannot call bluff on his own cards when one of them choses to pass
catchSelfBluff = False

# When the a new round starts
newRound = True

# To ensure that after two passes a new round starts
twoPasses = 2

# To keep a track of all the cards in the pile for each round
pile_cards = []

# To keep a track of only the most recently played cards
top_cards = []

# To keep a track of the rank called for the current round
current_round_rank = ""

# To keep a track of wether you are implementing strategy in the current round
strategy = False

# To keep a track of the current step of strategy ( strategy has 2 steps )
strategy_step = 1

## Exception Handing for Input

In [7]:
def safe_int_input(text):
  while True:
    try:
      return int(input(text))
    except:
      print("Invalid input. Please enter an integer.")

## Removing Cards Functions

In [8]:
def remove(hand, top_cards , pile_cards, maxFreqRank, maxFreqRankCount):

  index_to_remove = []

  for index,value in enumerate(hand):
    if(value == maxFreqRank):
      index_to_remove.append(index)

  for index in reversed(index_to_remove):
    pile_cards.append(hand[index])
    top_cards.append(hand[index])
    hand.pop(index)

## Removing Random Cards Function

In [9]:
def remove_random(hand, top_cards , pile_cards, newRound, current_round_rank, maxFreqRankCount):

  total_cards_removed = 0
  index_to_remove = []

  current_round_rank_cards = []


  for index,value in enumerate(hand):
    if(value == current_round_rank):
      current_round_rank_cards.append(value)
      hand.pop(index)

  while(total_cards_removed != maxFreqRankCount):

    minFreqRank = min(hand, key=hand.count)
    minFreqRankCount = hand.count(minFreqRank)

    for index,value in enumerate(hand):
      if(value == minFreqRank):
        index_to_remove.append(index)


    for index in reversed(index_to_remove):
      pile_cards.append(hand[index])
      top_cards.append(hand[index])
      hand.pop(index)
      total_cards_removed += 1

      if(total_cards_removed == maxFreqRankCount):
        break

    index_to_remove = []

  for value in current_round_rank_cards:
    hand.append(value)

## Honesty Function

In [10]:
def honesty(hand, top_cards, pile_cards, newRound, current_round_rank):

  if(newRound):
    maxFreqRank = max(hand, key=hand.count)
    maxFreqRankCount = hand.count(maxFreqRank)

  else:
    maxFreqRank = current_round_rank
    maxFreqRankCount = hand.count(maxFreqRank)

  remove(hand, top_cards , pile_cards, maxFreqRank, maxFreqRankCount)

  if(newRound):
    print("\nAgent called ", maxFreqRankCount, " of ", maxFreqRank)
  else:
    print("\nAgent added more ", maxFreqRankCount, " of ", maxFreqRank)

  return maxFreqRank, maxFreqRankCount

## Strategize Function

In [11]:
def strategize(hand, top_cards, pile_cards, newRound, current_round_rank, strategy, strategy_step):

  if(newRound and strategy_step == 1):
    maxFreqRank = max(hand, key=hand.count)

    length_of_hand = len(hand) - hand.count(maxFreqRank)

    if(length_of_hand == 1):
      maxFreqRankCount = 1
    else:
      maxFreqRankCount = random.randint(1,2)

    remove_random(hand, top_cards , pile_cards, newRound, maxFreqRank, maxFreqRankCount)
    print("\nAgent called ", maxFreqRankCount, " of ", maxFreqRank)
    strategy = True
    strategy_step += 1

  elif(not newRound and strategy_step == 1):
    maxFreqRank = current_round_rank

    length_of_hand = len(hand) - hand.count(maxFreqRank)

    if(length_of_hand == 1):
      maxFreqRankCount = 1
    else:
      maxFreqRankCount = random.randint(1,2)

    remove_random(hand, top_cards , pile_cards, newRound, current_round_rank, maxFreqRankCount)
    print("\nAgent added more ", maxFreqRankCount, " of ", maxFreqRank)
    strategy = True
    strategy_step += 1

  elif(strategy_step == 2):
    maxFreqRank = current_round_rank
    maxFreqRankCount = hand.count(maxFreqRank)
    remove(hand, top_cards , pile_cards, maxFreqRank, maxFreqRankCount)
    print("\nAgent added more ", maxFreqRankCount, " of ", maxFreqRank)
    strategy = False
    strategy_step = 0


  return maxFreqRank, maxFreqRankCount, strategy, strategy_step

## Lies Function

In [12]:
def lies(hand, top_cards, pile_cards, current_round_rank):

  maxFreqRank = current_round_rank
  maxFreqRankCount = random.randint(1,2)

  remove_random(hand, top_cards , pile_cards, False, current_round_rank, maxFreqRankCount)

  print("\nAgent added more ", maxFreqRankCount, " of ", maxFreqRank)

  return maxFreqRank, maxFreqRankCount

## Print User Options



In [13]:
def printHand(hand):
  print("\nYour hand ( ", len(hand),  " cards ): \n")

  for index, value in enumerate(hand):
    print("Index ", index, " -> ", value)

## User Input for Calling Cards

In [14]:
def user_calls_cards(hand, top_cards, pile_cards, newRound, current_round_rank):

  printHand(hand)

  maxFreqRank = current_round_rank

  if(newRound):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
    maxFreqRank = input("\nEnter the rank of the card (first letter capital): ")

    while maxFreqRank not in ranks:
      print("\nWrong Card Rank")
      print("\nAvailable Ranks -> ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']")
      maxFreqRank = input()

  maxFreqRankCount = safe_int_input("\nEnter the total number to call: ")

  while maxFreqRankCount <= 0:
    print("\nNumber can't be less than 1")
    maxFreqRankCount = safe_int_input("\nEnter the total number to call: ")

  index_to_remove = []

  while len(index_to_remove) != maxFreqRankCount:
    selected_index = safe_int_input("\nEnter the index of the cards you want to chose for the call: ")
    if(selected_index < 0 or selected_index > len(hand) - 1 or selected_index in index_to_remove):
      print("\nTry Again")
    else:
      index_to_remove.append(selected_index)

  index_to_remove.sort()

  for index in reversed(index_to_remove):
    pile_cards.append(hand[index])
    top_cards.append(hand[index])
    hand.pop(index)

  if(newRound):
    print("\nUser called ",maxFreqRankCount, " of ", maxFreqRank, "'s")
  else:
    print("\nUser added more ",maxFreqRankCount, " of ", maxFreqRank, "'s")

  return maxFreqRank, maxFreqRankCount

## Calling Function

In [15]:
def calling(hand, top_cards, pile_cards, isUserTurn, newRound, current_round_rank, strategy, strategy_step):

  if(isUserTurn and newRound):
    print("\nUser chose to play")
  elif(isUserTurn and not newRound):
    print("\nUser chose to continue playing")
  elif(not isUserTurn and newRound):
    print("\nAgent chose to play")
  else:
    print("\nAgent chose to continue playing")

  top_cards.clear()

  if(isUserTurn):
    maxFreqRank, maxFreqRankCount = user_calls_cards(hand, top_cards, pile_cards, newRound, current_round_rank)

    if(not newRound):
      maxFreqRank = current_round_rank

  else:
    if(strategy):
      print("Agent chose to strategize - 1")
      maxFreqRank, maxFreqRankCount, strategy, strategy_step = strategize(hand, top_cards, pile_cards, newRound, current_round_rank, strategy, strategy_step)

    elif(newRound):
      distinct_values = len(set(hand))

      if(distinct_values == 1):
        maxFreqRank, maxFreqRankCount = honesty(hand, top_cards , pile_cards, newRound, current_round_rank)
      else:
        option = random.randint(0,1)

        if(option == 0):
          print("Agent chose to play honestly - 1")
          maxFreqRank, maxFreqRankCount = honesty(hand, top_cards , pile_cards, newRound, current_round_rank)
        else:
          print("Agent chose to strategize - 2")
          maxFreqRank, maxFreqRankCount, strategy, strategy_step = strategize(hand, top_cards, pile_cards, newRound, current_round_rank, strategy, strategy_step)

    else:
      hasRank = True if current_round_rank in hand else False

      if(hasRank):
        distinct_values = len(set(hand))

        if(distinct_values == 1):
          maxFreqRank, maxFreqRankCount = honesty(hand, top_cards , pile_cards, newRound, current_round_rank)
        else:
          option = random.randint(0,1)

          if(option == 0):
            print("Agent chose to play honestly - 2")
            maxFreqRank, maxFreqRankCount = honesty(hand, top_cards , pile_cards, newRound, current_round_rank)
          else:
            print("Agent chose to strategize - 3")
            maxFreqRank, maxFreqRankCount, strategy, strategy_step = strategize(hand, top_cards, pile_cards, newRound, current_round_rank, strategy_step)

      else:
        print("Agent chose to lie - 1")
        maxFreqRank, maxFreqRankCount = lies(hand, top_cards, pile_cards, current_round_rank)

  return maxFreqRank, maxFreqRankCount, False, strategy, strategy_step


## Choice Function

In [16]:
def choice(isUserTurn, catchSelfBluff, newRound):

  if(newRound):
    option = 1;

  elif(isUserTurn and catchSelfBluff):
    print("\nYou can chose either one of the following\n1 -> Play Cards \n3 -> Pass")
    option = safe_int_input("\nEnter your choice: ")
    while(option < 1 or option > 3 or option == 2):
      print("\nWrong Choice")
      option = safe_int_input("\nEnter your choice: ")

  elif(isUserTurn and not catchSelfBluff):
    print("\nYou can chose either one of the following\n1 -> Play Cards \n2 -> Call Bluff \n3 -> Pass")
    option = safe_int_input("\nEnter your choice: ")
    while(option < 1 or option > 3):
      print("\nWrong Choice")
      option = safe_int_input("\nEnter your choice: ")

  elif(not isUserTurn and catchSelfBluff):
    option = random.choice([1,3])

  else:
    option = random.randint(1,3)

  return option, False

## Bluff Function

In [17]:
def bluff(user_hand, agent_hand, isUserTurn, top_cards, pile_cards, current_round_rank):

  isBluffing = False
  for value in top_cards:
    if(value != current_round_rank):
      isBluffing = True
      break

  # If the user calls the bluff & it is a bluff
  if(isBluffing and isUserTurn):
    agent_hand += pile_cards
    print("\nIt is a bluff")

  # If the user calls the bluff & it is not a bluff
  elif(not isBluffing and isUserTurn):
    user_hand += pile_cards
    print("\nIt is not a bluff")

  # If the agent calls the bluff & it is a bluff
  elif(isBluffing and not isUserTurn):
    user_hand += pile_cards
    print("\nIt is a bluff")

  # If the agent calls the bluff & it is not a bluff
  elif(not isBluffing and not isUserTurn):
    agent_hand += pile_cards
    print("\nIt is not a bluff")

  pile_cards.clear()

  return True if isBluffing else False

## Main Logic

In [18]:
while(len(user_hand) != 0 and len(agent_hand) != 0):

  if(newRound):
    print("\nNew Round !!!")


  if(isUserTurn):

    option, catchSelfBluff = choice(isUserTurn, catchSelfBluff, newRound)

    if(option == 1):
      maxFreqRank, maxFreqRankCount, newRound, strategy, strategy_step = calling(user_hand, top_cards, pile_cards, isUserTurn, newRound, current_round_rank, strategy, strategy_step)

      current_round_rank = maxFreqRank

      twoPasses = 2

    elif(option == 2):
      bluff(user_hand, agent_hand, isUserTurn, top_cards, pile_cards, current_round_rank)

      newRound = True

      twoPasses = 2

      strategy = False
      strategy_step = 1

    else:
      print("\nUser chose to pass")

      catchSelfBluff = True

      twoPasses -= 1

    isUserTurn = False

  else:

    option, catchSelfBluff = choice(isUserTurn, catchSelfBluff, newRound)

    if(strategy):
      option = 1

    if(option == 1):
      maxFreqRank, maxFreqRankCount, newRound, strategy, strategy_step = calling(agent_hand, top_cards, pile_cards, isUserTurn, newRound, current_round_rank, strategy, strategy_step)

      current_round_rank = maxFreqRank

      twoPasses = 2

    elif(option == 2):
      bluff(user_hand, agent_hand, isUserTurn, top_cards, pile_cards, current_round_rank)

      newRound = True

      twoPasses = 2

    else:
      print("\nAgent chose to pass")

      catchSelfBluff = True

      twoPasses -= 1

    isUserTurn = True

  if(twoPasses == 0):
    newRound = True
    twoPasses = 2


if(len(user_hand) == 0):
  print("\nUser Won")
elif(len(agent_hand) == 0):
  print("\nAgent Won")

-----------------------------------Debug---------------------------------------
User Hand:  ['King', '4', 'Jack', '6', 'Queen', 'Jack', '9']
Agent Hand:  ['8', '10', '8', '6', '3', '4', '9']
Pile Cards:  []
Top Cards:  []
Current Round Rank:  
-----------------------------------Debug---------------------------------------

New Round !!!

Agent chose to play
Agent chose to strategize - 2

Agent called  2  of  8
-----------------------------------Debug---------------------------------------
User Hand:  ['King', '4', 'Jack', '6', 'Queen', 'Jack', '9']
Agent Hand:  ['3', '4', '9', '8', '8']
Pile Cards:  ['10', '6']
Top Cards:  ['10', '6']
Current Round Rank:  8
-----------------------------------Debug---------------------------------------

You can chose either one of the following
1 -> Play Cards 
2 -> Call Bluff 
3 -> Pass

Enter your choice: 3

User chose to pass
-----------------------------------Debug---------------------------------------
User Hand:  ['King', '4', 'Jack', '6', 'Queen

KeyboardInterrupt: Interrupted by user